In [45]:
import numpy as np
import pandas as pd
import sympy as sp

In [46]:
# df = extract(key)
df = pd.read_excel('Pórtico 1.xlsx', header=None)
df

# número de nós
# x / y / restrição em x / restrição em y / restrição momento
# número de barras
# nó inicial / nó final / módulo de elasticidade / área / momento de inércia
# número de cargas em nós
# nó / valor da carga x / valor da carga y / valor do momento
# número de cargas em barras
# barra / direção / inicio da carga / fim da carga / valor da carga no inicio / valor da carga no fim

,0,1,2,3,4,5
0,3,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,1.0,1.0,1.0
2,2,0.0,6.0,0.0,0.0,0.0
3,3,4.0,6.0,1.0,1.0,1.0
4,2,NaN,NaN,NaN,NaN,NaN
5,1,1.0,2.0,1.0,1.0,1.0
6,2,2.0,3.0,1.0,1.0,1.0
7,2,NaN,NaN,NaN,NaN,NaN
8,2,10.0,0.0,0.0,NaN,NaN
9,2,0.0,-6.0,0.0,NaN,NaN


In [47]:
class node:
	def __init__(self, x, y, constraints):
		self.x = x
		self.y = y
		self.constraints = constraints
		self.loads = [0, 0, 0]

	def add_load(self, load, direction):
		self.loads[direction] += load

	def __str__(self):
		return f'Node - x: {self.x}, y: {self.y}, constraints: {self.constraints}'

In [48]:
class element:
	def __init__(self, node1, node2, E, A, I):
		self.node1 = node1
		self.node2 = node2
		self.L = np.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)
		self.E = E
		self.A = A
		self.I = I
		self.rotation = -np.rad2deg(np.arctan((node2.y - node1.y) / (node2.x - node1.x)))

	def __str__(self):
		return f'Element - node1: {self.node1}, node2: {self.node2}, L: {self.L}, E: {self.E}, A: {self.A}, I: {self.I}, rotation: {self.rotation}'


In [49]:
class structure:
	def __init__(self, df):
		self.nodes = {}
		self.elements = {}
		self.freedom_degrees = 0

		# leitura dos nós
		pos = 0
		n_nodes = df.iloc[pos, 0]
		pos = pos + 1

		for _ in range(1, n_nodes+1):
			self.nodes[df.iloc[pos, 0]] = node(df.iloc[pos, 1], df.iloc[pos, 2], [df.iloc[pos, 3], df.iloc[pos, 4], df.iloc[pos, 5]])
			pos += 1
		
		# leitura dos elementos
		n_elements = df.iloc[pos, 0]
		pos = pos + 1

		for _ in range(1, n_elements+1):
			self.elements[df.iloc[pos, 0]] = element(self.nodes[df.iloc[pos, 1]], self.nodes[df.iloc[pos, 2]], df.iloc[pos, 3], df.iloc[pos, 4], df.iloc[pos, 5])
			pos += 1

		# leitura das cargas nos nós
		n_loads = df.iloc[pos, 0]
		pos = pos + 1

		for _ in range(1, n_loads+1):
			self.nodes[df.iloc[pos, 0]].add_load(df.iloc[pos, 1], 0)
			self.nodes[df.iloc[pos, 0]].add_load(df.iloc[pos, 2], 1)
			self.nodes[df.iloc[pos, 0]].add_load(df.iloc[pos, 3], 2)
			pos += 1

		# leitura das cargas nas barras (já convertendo para cargas nos nós)
		n_elements_loads = df.iloc[pos, 0]
		pos = pos + 1

		for _ in range(1, n_elements_loads+1):
			if df.iloc[pos, 1] == 0:
				if df.iloc[pos, 2] != df.iloc[pos, 3]:
					print('There is no distributed load on the axial direction')
				else:
					if df.iloc[pos, 4] != df.iloc[pos, 5]:
						print('A single load cannot have two different values')
					else:
						current_element = self.elements[df.iloc[pos, 0]]
						l = current_element.L
						a = df.iloc[pos, 2] * l
						b = l - a
						P = df.iloc[pos, 4]
						current_element.node1.add_load(- P * b / l, 0)
						current_element.node2.add_load(- P * a / l, 0)
				pass
			elif df.iloc[pos, 1] == 1:
				if df.iloc[pos, 4] != df.iloc[pos, 5]:
					if df.iloc[pos, 4] != 0 and df.iloc[pos, 5] != 0:
						print('One of the values of the distributed linear load must be zero')
					else:
						if not (df.iloc[pos, 2] == 0 and df.iloc[pos, 3] == 1) and not (df.iloc[pos, 2] == 1 and df.iloc[pos, 3] == 0):
							print('Linear distributed load cannot be in the middle of the element')
						else:
							if df.iloc[pos, 4] == 0:
								current_element = self.elements[df.iloc[pos, 0]]
								l = current_element.L
								q = df.iloc[pos, 5]
								current_element.node1.add_load(q * l**2 / 30, 2)
								current_element.node2.add_load(-q * l**2 / 20, 2)
							else:
								current_element = self.elements[df.iloc[pos, 0]]
								l = current_element.L
								q = df.iloc[pos, 4]
								current_element.node1.add_load(q * l**2 / 20, 2)
								current_element.node2.add_load(-q * l**2 / 30, 2)
				else:
					if df.iloc[pos, 2] == df.iloc[pos, 3]:
						print(df.iloc[pos, 2])
						print(df.iloc[pos, 3])
						print('carga pontual no meio')
						current_element = self.elements[df.iloc[pos, 0]]
						l = current_element.L
						P = df.iloc[pos, 4]
						a = df.iloc[pos, 2] * l
						b = l - a
						current_element.node1.add_load(P * a * b**2 / l**2, 2)
						current_element.node2.add_load(-P * a**2 * b / l**2, 2)
					else:
						print(df.iloc[pos, 2])
						print(df.iloc[pos, 3])
						print(' carga distriboi')
						current_element = self.elements[df.iloc[pos, 0]]
						l = current_element.L
						q = df.iloc[pos, 4]
						aux = abs(df.iloc[pos, 2] - df.iloc[pos, 3])
						c = aux * l
						a = (df.iloc[pos, 2] + aux/2) * l
						b = l - a
						current_element.node1.add_load(q * c / 12 / l**2 * (12 * a * b**2 + c**2 * (l - 3 * b)), 2)
						current_element.node2.add_load(-q * c / 12 / l**2 * (12 * a**2 * b + c**2 * (l - 3 * a)), 2)
			elif df.iloc[pos, 1] == 2:
				if df.iloc[pos, 2] != df.iloc[pos, 3]:
					print('There is no distributed moment')
				else:
					if df.iloc[pos, 4] != df.iloc[pos, 5]:
						print('A single moment cannot have two different values')
					else:
						current_element = self.elements[df.iloc[pos, 0]]
						l = current_element.L
						a = df.iloc[pos, 2] * l
						b = l - a
						M = df.iloc[pos, 4]
						current_element.node1.add_load((M * b / l) * (2 - 3 * b / l), 2)
						current_element.node2.add_load((M * a / l) * (2 - 3 * a / l), 2)
				pass
			else:
				print('Invalid load direction')
			pos += 1

		# cálculo dos graus de liberdade
		for key, value in self.nodes.items():
			for i in range(3):
				if value.constraints[i] == 0:
					self.freedom_degrees += 1

	def __str__(self):
		s = 'Structure\n'	
		s += '... Nodes:\n'
		for key, value in self.nodes.items():
			s += f'....... {key}: {value}\n'
		s += '... Elements:\n'
		for key, value in self.elements.items():
			s += f'....... {key}: {value}\n'
		s += f'... Freedom degrees: {self.freedom_degrees}\n'
		return s

In [50]:
s = structure(df)

for key, value in s.nodes.items():
	print(key, value.loads)

print(s)

1 [0, 0, 0]
2 [10.0, -6.0, 0.0]
3 [0, 0, 0]
Structure
... Nodes:
....... 1: Node - x: 0.0, y: 0.0, constraints: [1.0, 1.0, 1.0]
....... 2: Node - x: 0.0, y: 6.0, constraints: [0.0, 0.0, 0.0]
....... 3: Node - x: 4.0, y: 6.0, constraints: [1.0, 1.0, 1.0]
... Elements:
....... 1: Element - node1: Node - x: 0.0, y: 0.0, constraints: [1.0, 1.0, 1.0], node2: Node - x: 0.0, y: 6.0, constraints: [0.0, 0.0, 0.0], L: 6.0, E: 1.0, A: 1.0, I: 1.0, rotation: -90.0
....... 2: Element - node1: Node - x: 0.0, y: 6.0, constraints: [0.0, 0.0, 0.0], node2: Node - x: 4.0, y: 6.0, constraints: [1.0, 1.0, 1.0], L: 4.0, E: 1.0, A: 1.0, I: 1.0, rotation: -0.0
... Freedom degrees: 3



C:\Users\joaos\AppData\Local\Temp\ipykernel_15000\664103471.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  self.rotation = -np.rad2deg(np.arctan((node2.y - node1.y) / (node2.x - node1.x)))


In [51]:
from sympy import symbols
from sympy.abc import E, A, I, L, theta

K = [[ A*E/L,            0,           0,      -A*E/L,            0,           0],
	 [     0,  12*E*I/L**3,  6*E*I/L**2,           0, -12*E*I/L**3,  6*E*I/L**2],
	 [     0,   6*E*I/L**2,     4*E*I/L,           0,  -6*E*I/L**2,     2*E*I/L],
	 [-A*E/L,            0,           0,       A*E/L,            0,           0],
	 [     0, -12*E*I/L**3, -6*E*I/L**2,           0,  12*E*I/L**3, -6*E*I/L**2],
	 [     0,   6*E*I/L**2,     2*E*I/L,           0,  -6*E*I/L**2,     4*E*I/L]]

# tensor of rotation
T = [[ sp.cos(theta), sp.sin(theta), 0,              0,             0, 0],
	 [-sp.sin(theta), sp.cos(theta), 0,              0,             0, 0],
	 [             0,             0, 1,              0,             0, 0],
	 [             0,             0, 0,  sp.cos(theta), sp.sin(theta), 0],
	 [             0,             0, 0, -sp.sin(theta), sp.cos(theta), 0],
	 [             0,             0, 0,              0,             0, 1]]

display(sp.Matrix(K))

Matrix([
[ A*E/L,            0,           0, -A*E/L,            0,           0],
[     0,  12*E*I/L**3,  6*E*I/L**2,      0, -12*E*I/L**3,  6*E*I/L**2],
[     0,   6*E*I/L**2,     4*E*I/L,      0,  -6*E*I/L**2,     2*E*I/L],
[-A*E/L,            0,           0,  A*E/L,            0,           0],
[     0, -12*E*I/L**3, -6*E*I/L**2,      0,  12*E*I/L**3, -6*E*I/L**2],
[     0,   6*E*I/L**2,     2*E*I/L,      0,  -6*E*I/L**2,     4*E*I/L]])

In [52]:
K_locals = {}

# calculate local matrixes for each element
for key, value in s.elements.items():
	# calculate local matrix
	K_local = sp.Matrix(K).subs([(E, value.E), (A, value.A), (I, value.I), (L, value.L)])
	# substitute theta
	rot = sp.Matrix(T).subs([(theta, np.deg2rad(value.rotation))])	
	# rotate local matrix using tensor of rotation
	K_local = sp.Matrix(rot).T * K_local * sp.Matrix(rot)
	# put 0 in the matrix where the node is fixed
	left_constraints = value.node1.constraints
	right_constraints = value.node2.constraints
	for i in range(3):
		if left_constraints[i] == 1:
			for j in range(6):
				K_local[j, i] = 0
				K_local[i, j] = 0
		if right_constraints[i] == 1:
			for j in range(6):
				K_local[j, 3+i] = 0
				K_local[3+i, j] = 0
	# add local matrix to dictionary
	K_locals[key] = K_local
	display(sp.Matrix(K_local))

Matrix([
[0, 0, 0,                     0,                     0,                     0],
[0, 0, 0,                     0,                     0,                     0],
[0, 0, 0,                     0,                     0,                     0],
[0, 0, 0,    0.0555555555555556, -6.80359332859641e-18,    -0.166666666666667],
[0, 0, 0, -6.80359332859641e-18,     0.166666666666667, -1.02053899928946e-17],
[0, 0, 0,    -0.166666666666667, -1.02053899928946e-17,     0.666666666666667]])

Matrix([
[0.25,      0,     0, 0, 0, 0],
[   0, 0.1875, 0.375, 0, 0, 0],
[   0,  0.375,   1.0, 0, 0, 0],
[   0,      0,     0, 0, 0, 0],
[   0,      0,     0, 0, 0, 0],
[   0,      0,     0, 0, 0, 0]])

In [53]:
# calculate global matrix
#number of elements * 6
size = 3 + len(s.elements) * 3
K_global = sp.zeros(size, size)

# calculate global matrix
for key, value in s.elements.items():
	# get local matrix
	K_local = K_locals[key]
	# get global indexes
	index = (key - 1) * 3

	# add local matrix to global matrix
	for i in range(6):
		for j in range(6):
			K_global[index + i, index + j] += K_local[i, j]

display(sp.Matrix(K_global))

Matrix([
[0, 0, 0,                     0,                     0,                  0, 0, 0, 0],
[0, 0, 0,                     0,                     0,                  0, 0, 0, 0],
[0, 0, 0,                     0,                     0,                  0, 0, 0, 0],
[0, 0, 0,     0.305555555555556, -6.80359332859641e-18, -0.166666666666667, 0, 0, 0],
[0, 0, 0, -6.80359332859641e-18,     0.354166666666667,              0.375, 0, 0, 0],
[0, 0, 0,    -0.166666666666667,                 0.375,   1.66666666666667, 0, 0, 0],
[0, 0, 0,                     0,                     0,                  0, 0, 0, 0],
[0, 0, 0,                     0,                     0,                  0, 0, 0, 0],
[0, 0, 0,                     0,                     0,                  0, 0, 0, 0]])

In [54]:
# create a symbol for each node
u = []
for i in range(s.nodes.__len__()):
	u.append(sp.symbols(f'u{i}_x'))
	u.append(sp.symbols(f'u{i}_y'))
	u.append(sp.symbols(f'u{i}_m'))

# matrix of displacements (with sympy symbols)
u = sp.Matrix(u)

display(sp.Matrix(u))

# matrix of forces (get from nodes if is not fixed)
F = sp.zeros(size, 1)
for key, value in s.nodes.items():
	index = key - 1
	if value.constraints[0] == 0:
		F[index * 3, 0] = value.loads[0]
	if value.constraints[1] == 0:
		F[index * 3 + 1, 0] = value.loads[1]
	if value.constraints[2] == 0:
		F[index * 3 + 2, 0] = value.loads[2]

display(sp.Matrix(F))

Matrix([
[u0_x],
[u0_y],
[u0_m],
[u1_x],
[u1_y],
[u1_m],
[u2_x],
[u2_y],
[u2_m]])

Matrix([
[   0],
[   0],
[   0],
[10.0],
[-6.0],
[   0],
[   0],
[   0],
[   0]])

In [55]:
# solve - F + K*u = 0
eq = - F + K_global * u
display(eq)

# solve for u
u = sp.solve(eq, u)
display(u)

Matrix([
[                                                                                  0],
[                                                                                  0],
[                                                                                  0],
[-0.166666666666667*u1_m + 0.305555555555556*u1_x - 6.80359332859641e-18*u1_y - 10.0],
[              0.375*u1_m - 6.80359332859641e-18*u1_x + 0.354166666666667*u1_y + 6.0],
[                        1.66666666666667*u1_m - 0.166666666666667*u1_x + 0.375*u1_y],
[                                                                                  0],
[                                                                                  0],
[                                                                                  0]])

{u1_m: 10.0173913043478, u1_x: 38.1913043478261, u1_y: -27.5478260869565}